# Little Lemon Booking System

This notebook contains the client-side code for Little Lemon's booking system.

## Setup and Configuration

### Task 1: Verify Python Installation

First, let's verify that Python is installed correctly.

In [ ]:
# Check Python version
import sys
print(f"Python version: {sys.version}")
print(f"Python version info: {sys.version_info}")

# Verify Python 3.x
if sys.version_info.major >= 3:
    print("✅ Python 3.x is installed correctly")
else:
    print("❌ Python 3.x is required. Please install Python 3.")

### Task 2: Install Jupyter and Dependencies

Install required packages if not already installed.

In [ ]:
# Install MySQL Connector (Task 3, Step 1)
!pip install mysql-connector-python

### Task 3: Database Connection Setup

#### Step 2: Import the Connector

In [ ]:
# Import MySQL connector (Task 3, Step 2)
import mysql.connector as connector
print("✅ MySQL connector imported successfully")

#### Step 3: Test Database Connection

In [ ]:
# Test database connection (Task 3, Step 3)
# Update these credentials with your MySQL username and password
connection = connector.connect(
    user="your_user_name",  # Replace with your MySQL username
    password="your_password",  # Replace with your MySQL password
    host="localhost",
    database="LittleLemonDB"
)

# Verify connection
if connection.is_connected():
    print("✅ Successfully connected to MySQL database")
    
    # Get database info
    db_info = connection.get_server_info()
    print(f"MySQL Server version: {db_info}")
    
    # Get database name
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    database_name = cursor.fetchone()[0]
    print(f"Connected to database: {database_name}")
    
    cursor.close()
    connection.close()
    print("✅ Connection closed successfully")
else:
    print("❌ Failed to connect to MySQL database")

## Exercise: Add Query Functions

This section implements query functionality to interact with the database.

### Task One: Connect to Database and Create Cursor

Import the connector module, enter your user details, and connect with the database. Then create a cursor instance to pass queries and return results.

In [ ]:
# Task One: Import connector and connect to database
import mysql.connector as connector

# Connect to the database using the db parameter
# Update these credentials with your MySQL username and password
connection = connector.connect(
    user="your_username",  # Replace with your MySQL username
    password="your_password",  # Replace with your MySQL password
    db="LittleLemonDB"  # Database name (using 'db' parameter instead of 'database')
)

# Verify connection
if connection.is_connected():
    print("✅ Successfully connected to LittleLemonDB database")
    
    # Instantiate a cursor to pass queries and return results
    cursor = connection.cursor()
    print("✅ Cursor created successfully")
    
    # Note: Connection and cursor are now ready for queries
else:
    print("❌ Failed to connect to database")

### Task Two: Show All Tables

Execute a test query to show all tables within the database to ensure the connection is working properly.

In [ ]:
# Task Two: Query to show all tables in the database
show_tables_query = "SHOW tables"

# Execute the query on the cursor
cursor.execute(show_tables_query)

# Fetch all results using cursor.fetchall()
results = cursor.fetchall()

# Print the results
print("Tables in LittleLemonDB database:")
print(results)

# Display results in a more readable format
print("\nFormatted table list:")
for table in results:
    print(f"  - {table[0]}")

### Task Three: Query with Table JOIN

Return specific details from the database: full name and contact details for every customer that has placed an order greater than $60 for a promotional campaign.

**Steps:**
1. Identify which tables are required (Orders and Customers)
2. Select the required attributes (full name, contact details, bill amount)
3. Join the tables on a common attribute (CustomerID)
4. Filter data using WHERE clause (TotalCost > 60)

In [ ]:
# Task Three: Query with JOIN to get customer details for orders > $60

# Step One: Identify tables and select attributes
# - Orders table: TotalCost (bill amount)
# - Customers table: FirstName, LastName (full name), Email, Phone (contact details)

# Step Two: Specify FROM and JOIN
# Join Orders and Customers tables on CustomerID (common attribute)

# Step Three: Add WHERE clause to filter orders > $60

# Complete query
join_query = """
SELECT 
    CONCAT(c.FirstName, ' ', c.LastName) AS FullName,
    c.Email,
    c.Phone,
    o.TotalCost AS BillAmount
FROM 
    Orders o
INNER JOIN 
    Customers c ON o.CustomerID = c.CustomerID
WHERE 
    o.TotalCost > 60
ORDER BY 
    o.TotalCost DESC
"""

# Execute the query
cursor.execute(join_query)

# Fetch all results
results = cursor.fetchall()

# Print the results
print("Customers with orders greater than $60:")
print("-" * 80)

# Display results in a formatted way
if results:
    print(f"{'Full Name':<30} {'Email':<30} {'Phone':<15} {'Bill Amount':<12}")
    print("-" * 80)
    for row in results:
        full_name, email, phone, bill_amount = row
        print(f"{full_name:<30} {email:<30} {phone:<15} ${bill_amount:<11.2f}")
    print(f"\nTotal customers found: {len(results)}")
else:
    print("No customers found with orders greater than $60")

### Clean Up: Close Connection

Always close the cursor and connection when done to free up resources.

In [ ]:
# Close cursor and connection
if connection.is_connected():
    cursor.close()
    connection.close()
    print("✅ Cursor and connection closed successfully")

## Alternative: Using Configuration File

For better security and reusability, you can use the config.py file for database credentials.

In [ ]:
# Alternative approach using config.py
import sys
import os

# Add parent directory to path to import config
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

try:
    from config import DB_CONFIG
    import mysql.connector as connector
    
    # Connect using config
    connection = connector.connect(**DB_CONFIG)
    
    if connection.is_connected():
        print("✅ Connected using config.py")
        cursor = connection.cursor()
        
        # Show tables
        cursor.execute("SHOW tables")
        tables = cursor.fetchall()
        print(f"\nTables in database: {len(tables)}")
        for table in tables:
            print(f"  - {table[0]}")
        
        # Query customers with orders > $60
        join_query = """
        SELECT 
            CONCAT(c.FirstName, ' ', c.LastName) AS FullName,
            c.Email,
            c.Phone,
            o.TotalCost AS BillAmount
        FROM Orders o
        INNER JOIN Customers c ON o.CustomerID = c.CustomerID
        WHERE o.TotalCost > 60
        ORDER BY o.TotalCost DESC
        """
        
        cursor.execute(join_query)
        results = cursor.fetchall()
        
        print(f"\nCustomers with orders > $60: {len(results)}")
        for row in results:
            print(f"  {row[0]} - {row[1]} - ${row[3]:.2f}")
        
        cursor.close()
        connection.close()
        print("\n✅ Connection closed")
        
except ImportError:
    print("⚠️  config.py not found. Update credentials in the cells above.")
except Exception as e:
    print(f"❌ Error: {e}")